In [2]:
import numpy as np
import dynesty
from scipy.stats import truncnorm
from astropy import units as u
from astropy import coordinates as coord
import matplotlib.pyplot as plt

# ===================== 常量 & 模型定义 =====================
G = 43018  # 引力常数 (kpc (km/s)^2 / 10^10 Msun)

def rotation_matrix_theta(theta):
    return np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])

def rotation_matrix_phi(phi):
    return np.array([
        [np.cos(phi), -np.sin(phi), 0],
        [np.sin(phi), np.cos(phi), 0],
        [0, 0, 1]
    ])

def Galactic_to_icrs(pos, vel):
    gc = coord.SkyCoord(
        x=pos[0] * u.kpc, y=pos[1] * u.kpc, z=pos[2] * u.kpc,
        v_x=vel[0] * u.km/u.s, v_y=vel[1] * u.km/u.s, v_z=vel[2] * u.km/u.s,
        frame='galactocentric',
        representation_type='cartesian',
        differential_type='cartesian',
        galcen_distance=8.122 * u.kpc,
        z_sun=20.8 * u.pc,
        galcen_v_sun=coord.CartesianDifferential([12.9, 245.6, 7.8] * u.km/u.s)
    )
    c = gc.transform_to(coord.ICRS)
    return (
        c.ra.value, c.dec.value, c.distance.value,
        c.pm_ra_cosdec.value, c.pm_dec.value, c.radial_velocity.value
    )

def orbit_model(params):
    a, e, M, eta, theta, phi = params
    T = np.sqrt(a**3 / (G * M))  # 轨道周期 (Gyr)
    t = T * (eta - e * np.sin(eta))  # 时间参数
    
    r = a * (1 - e * np.cos(eta))
    v_factor = np.sqrt(G * M / a)
    v_rad = v_factor * e * np.sin(eta) / (1 - e * np.cos(eta))
    v_tan = v_factor * np.sqrt(1 - e**2) / (1 - e * np.cos(eta))
    
    pos = rotation_matrix_theta(theta) @ rotation_matrix_phi(phi) @ np.array([r, 0, 0])
    vel = rotation_matrix_theta(theta) @ rotation_matrix_phi(phi) @ np.array([v_rad, v_tan, 0])
    
    ra, dec, distance, pmra, pmdec, vr = Galactic_to_icrs(pos, vel)
    return ra, dec, distance, pmra, pmdec, vr, t

# ===================== 观测数据 =====================
obs_data = {
    'ra': (10.68, 0.1),          # deg
    'dec': (41.27, 0.1),         # deg
    'distance': (761, 11),       # kpc
    'pmra': (46.9e-3, 10e-3),    # mas/yr
    'pmdec': (-29.1e-3, 10e-3),  # mas/yr
    'vr': (-301, 1),             # km/s
    'time': (13.8, 0.024)        # Gyr
}

# ===================== 嵌套采样核心 =====================
def prior_transform(u):
    """将单位立方体 [0,1]^6 转换为物理参数"""
    a_u, e_u, M_u, eta_u, theta_u, phi_u = u
    
    # 1. a: 截断正态分布 N(700, 100), a > 0
    a = truncnorm.ppf(a_u, (0 - 700)/100, np.inf, loc=700, scale=100)
    
    # 2. e: 1 - exp(-10u) 实现 ln(1-e) ~ U(-10,0)
    e = 1 - np.exp(-10 * e_u)
    e = np.clip(e, 0, 0.999)  # 防止 e=1
    
    # 3. M: 截断正态分布 N(400, 300), M > 0
    M = truncnorm.ppf(M_u, (0 - 400)/300, np.inf, loc=400, scale=300)
    
    # 4. 角度参数
    eta = 2 * np.pi * eta_u       # [0, 2π]
    theta = -np.pi/2 + np.pi * theta_u  # [-π/2, π/2]
    phi = np.pi * phi_u           # [0, π]
    
    return [a, e, M, eta, theta, phi]

def log_likelihood(params):
    """对数似然函数（包含数值保护）"""
    a, e, M, eta, theta, phi = params
    
    # 参数合法性检查
    if (a <= 0) or (M <= 0) or (e < 0) or (e >= 1):
        return -np.inf
    
    try:
        # 计算模型预测
        model_ra, model_dec, model_dist, model_pmra, model_pmdec, model_vr, t = orbit_model(params)
    except:
        return -np.inf
    
    # 计算 χ²
    chi2 = 0
    chi2 += ((model_ra - obs_data['ra'][0])/obs_data['ra'][1])**2
    chi2 += ((model_dec - obs_data['dec'][0])/obs_data['dec'][1])**2
    chi2 += ((model_dist - obs_data['distance'][0])/obs_data['distance'][1])**2
    chi2 += ((model_pmra - obs_data['pmra'][0])/obs_data['pmra'][1])**2
    chi2 += ((model_pmdec - obs_data['pmdec'][0])/obs_data['pmdec'][1])**2
    chi2 += ((model_vr - obs_data['vr'][0])/obs_data['vr'][1])**2
    chi2 += ((t - obs_data['time'][0])/obs_data['time'][1])**2
    
    return -0.5 * chi2




In [6]:
sampler = dynesty.NestedSampler(
    loglikelihood=log_likelihood,
    prior_transform=prior_transform,
    ndim=6,
    nlive=500,
    bound='multi',
    sample='auto',
    # 不再需要显式传递 live_points，采样器会自动生成
    # 但需要显式定义参数范围
    bootstrap=0,  # 禁用自动范围检测
)

# ===================== 手动设置参数范围 =====================
# 定义参数范围
param_bounds = [
    (0, 1500),        # a
    (0, 0.999),       # e
    (0, 1000),        # M
    (0, 2*np.pi),     # eta
    (-np.pi/2, np.pi/2),  # theta
    (0, np.pi)        # phi
]

# 通过动态更新范围确保参数不越界
sampler.live_update(
    update_interval=0.6,  # 更新频率
    first_update={"min_eff": 10, "min_ncall": 100},
    bounds=param_bounds  # 显式传递范围
)

# ===================== 运行采样 =====================
sampler.run_nested(
    print_progress=True,
    maxiter=10000,
    maxcall=100000,
    dlogz=0.1
)


AttributeError: 'MultiEllipsoidSampler' object has no attribute 'live_update'